# Stokes Gleichung (1D)

## Einführung

- Theorie und Gleichungen

## Problem und Lösung

- Poiseuille-Coutte Flow (1D)

In [ ]:
using Plots, ExtendableSparse

In [ ]:
# Model Parameter ------------------------------------------------------- #
M   =   (
    ymin        =   -400.0e3,           #   Tiefe [ m ]
    ymax        =   0.0e3,              
)
I   =   (
    vₓ₀         =   5/100/31536000,     #   Geschwindigkeit oben [ m/s ]
    η₀          =   1.0e21,             #   Viscosität [ Pa s ]
    ∂P∂x        =   -5.0e1,              #   horizontaler Druckgradient [ Pa/m ]
)
# ----------------------------------------------------------------------- #

In [ ]:
# Numerische Parameter -------------------------------------------------- #
NC  =   (
    y   =   100,        #   Anzahl der vertikalen Centroids
)
Δ   =   (
    y   =   (M.ymax-M.ymin)/NC.y,   #   Gitterauflösung
)
# ----------------------------------------------------------------------- #
# Speicherzuweisung ----------------------------------------------------- #
D   =   (
    vₓ  =   zeros(NC...),
    vₓₐ =   zeros(NC...),
    Δvₓ =   zeros(NC...)
)
# ----------------------------------------------------------------------- #
# Gitter ---------------------------------------------------------------- #
y   =   (
    c   =   LinRange(M.ymin+Δ.y/2,M.ymax-Δ.y/2,NC.y),
)
# ----------------------------------------------------------------------- #

In [ ]:
# Analytische Lösung ---------------------------------------------------- # 
@. D.vₓₐ    =   1.0/2.0/I.η₀ * I.∂P∂x * 
                    (y.c^2 + (M.ymax-M.ymin).*y.c) + 
                    I.vₓ₀*y.c/(M.ymax-M.ymin) + I.vₓ₀
# ----------------------------------------------------------------------- #

In [ ]:
# Randbedingungen ------------------------------------------------------- #
VBC     =   (
    type    = (S=:Dirichlet, N=:Dirichlet),
    val     = (S=0.0,N=I.vₓ₀)
)
# ----------------------------------------------------------------------- #

In [ ]:
# Lineares Gleichungssystem --------------------------------------------- #
Num     =   (Vx=1:NC.y,)
ndof    =   maximum(Num.Vx)
K       =   ExtendableSparseMatrix(ndof,ndof)
rhs     =   zeros(NC...)
# ----------------------------------------------------------------------- #
# Zusammenstellen der Koeffizientenmatrix ------------------------------- #
a       =     I.η₀ / Δ.y^2.0
b       =   - 2.0 * I.η₀ / Δ.y^2.0
rhs     .=  I.∂P∂x
for i = 1:NC.y
    # Gleichungsnummer ---
    ii  =   Num.Vx[i]
    # Stempel ---
    iN  =   ii + 1      #   Norden
    iC  =   ii          #   Zentral
    iS  =   ii - 1      #   Süden
    # Ränder ---
    # Falls ein Süd Index gebrauch wird ---
    inS    =  i==1    ? false  : true
    DirS   = (i==1    && VBC.type.S==:Dirichlet) ? 1. : 0.
    NeuS   = (i==1    && VBC.type.S==:Neumann  ) ? 1. : 0.
    # If an East index is required ---
    inN    =  i==NC.y ? false  : true
    DirN   = (i==NC.y && VBC.type.N==:Dirichlet) ? 1. : 0.
    NeuN   = (i==NC.y && VBC.type.N==:Neumann  ) ? 1. : 0.
    if inS K[ii,iS]    = a end
        K[ii,iC]       =   b + (NeuS + NeuN - DirS - DirN)*a
    if inN K[ii,iN]    = a end    
    # Änderung der rechten Seite ---
    rhs[i]      +=  a*VBC.val.S*Δ.y * NeuS - 
                        2*a*VBC.val.S * DirS - 
                        a*VBC.val.N*Δ.y * NeuN - 
                        2*a*VBC.val.N * DirN
end
# ----------------------------------------------------------------------- #

In [ ]:
# Lösung des Gleichungssystems ------------------------------------------ #
D.vₓ      .=   K \ rhs
# ----------------------------------------------------------------------- #
# Abweichung vom der analytischen Lösung -------------------------------- #
@. D.Δvₓ    =   ((D.vₓₐ - D.vₓ) / D.vₓₐ) * 100.0
# ----------------------------------------------------------------------- #

In [ ]:
# Visualisierung -------------------------------------------------------- #
q   =   plot(D.vₓ,y.c./1e3,label="numerical",
            xlabel="vₓ", ylabel="y [km]",
            title="Velocity Profile",
            xlim=(0,I.vₓ₀*1.5),ylim=(M.ymin/1e3,M.ymax/1e3),
            layout=(1,2),subplot=1)
plot!(q,D.vₓₐ,y.c./1e3,label="analytical",linestyle=:dash,
        subplot=1)
plot!(q,D.Δvₓ,y.c./1e3,label="",
        xlabel="Δvₓ",ylabel="z [km]",
        title="Error [ % ]",
        subplot=2)
display(q)
# ----------------------------------------------------------------------- #